In [ ]:
import csv
import sqlite3
import pytz
from datetime import datetime

In [1]:
from itineraryBuilder import *

In [2]:
#conn = sqlite3.connect('faa')
origin = 'ATL'
destination = 'DTW'
flight_date = '8/3/2022'

dne = '700'
dnl = '1000'
ane = '1'
anl = '2000'

df = queryFlights('faa', origin, destination, flight_date, dne, dnl, ane, anl); df

,FIRST_LEG_AIRLINE,FIRST_LEG_ORIG,FIRST_LEG_ORIG_CITY,FIRST_LEG_DEST,FIRST_LEG_DEST_CITY,FIRST_LEG_DATE,FIRST_LEG_FLIGHT_NUM,FIRST_LEG_DEP_TIME,FIRST_LEG_ARR_TIME,SECOND_LEG_AIRLINE,SECOND_LEG_ORIG,SECOND_LEG_ORIG_CITY,SECOND_LEG_DEST,SECOND_LEG_DEST_CITY,SECOND_LEG_DATE,SECOND_LEG_FLIGHT_NUM,SECOND_LEG_DEP_TIME,SECOND_LEG_ARR_TIME
0,DL,ATL,"Atlanta, GA",LEX,"Lexington, KY",8/3/2022,5660,910,1018,DL,LEX,"Lexington, KY",DTW,"Detroit, MI",8/3/2022,4932,1530,1648
1,DL,ATL,"Atlanta, GA",LEX,"Lexington, KY",8/3/2022,5660,910,1018,DL,LEX,"Lexington, KY",DTW,"Detroit, MI",8/3/2022,5637,1103,1225
2,DL,ATL,"Atlanta, GA",LEX,"Lexington, KY",8/3/2022,5660,910,1018,DL,LEX,"Lexington, KY",DTW,"Detroit, MI",8/3/2022,5827,615,736
3,DL,ATL,"Atlanta, GA",LEX,"Lexington, KY",8/3/2022,5660,910,1018,DL,LEX,"Lexington, KY",DTW,"Detroit, MI",8/4/2022,4932,1530,1648
4,DL,ATL,"Atlanta, GA",LEX,"Lexington, KY",8/3/2022,5660,910,1018,DL,LEX,"Lexington, KY",DTW,"Detroit, MI",8/4/2022,5637,1103,1225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,DL,ATL,"Atlanta, GA",ROC,"Rochester, NY",8/3/2022,2496,941,1141,DL,ROC,"Rochester, NY",DTW,"Detroit, MI",8/4/2022,5299,635,755
519,DL,ATL,"Atlanta, GA",TYS,"Knoxville, TN",8/3/2022,2530,820,908,DL,TYS,"Knoxville, TN",DTW,"Detroit, MI",8/3/2022,4812,730,909
520,DL,ATL,"Atlanta, GA",TYS,"Knoxville, TN",8/3/2022,2530,820,908,DL,TYS,"Knoxville, TN",DTW,"Detroit, MI",8/3/2022,4836,1240,1419
521,DL,ATL,"Atlanta, GA",TYS,"Knoxville, TN",8/3/2022,2530,820,908,DL,TYS,"Knoxville, TN",DTW,"Detroit, MI",8/4/2022,4812,730,909


In [3]:
df2 = getValidDestinations('faa', origin, flight_date); df2

,AIRPORT,CITY
0,DCA,"Washington, DC"
1,DTW,"Detroit, MI"
2,LGA,"New York, NY"
3,MSP,"Minneapolis, MN"
4,BOS,"Boston, MA"
...,...,...
169,TUL,"Tulsa, OK"
170,TWF,"Twin Falls, ID"
171,WYS,"West Yellowstone, MT"
172,CNY,"Moab, UT"


In [4]:
df = itineraryBuilder(df, tc=45); df

C:\Users\parso\OneDrive\Documents\Education\Data Analytics\CSE-6242_Data and Visual Analytics\Project\gatech-cse6242-spring2023-project\dataManagement\itineraryBuilder.py:245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RISK_MISSED_CONNECTION'] = dummyMLResult(num_rows)
C:\Users\parso\OneDrive\Documents\Education\Data Analytics\CSE-6242_Data and Visual Analytics\Project\gatech-cse6242-spring2023-project\dataManagement\itineraryBuilder.py:246: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NEXT_FLIG

,FIRST_LEG_AIRLINE,FIRST_LEG_ORIG,FIRST_LEG_ORIG_CITY,FIRST_LEG_DEST,FIRST_LEG_DEST_CITY,FIRST_LEG_DATE,FIRST_LEG_FLIGHT_NUM,FIRST_LEG_DEP_TIME,FIRST_LEG_ARR_TIME,SECOND_LEG_AIRLINE,...,FIRST_LEG_ARR_TIMESTAMP,SECOND_LEG_DEP_TIMESTAMP,SECOND_LEG_ARR_TIMESTAMP,overnight_bool_1,overnight_bool_2,CONNECT_TIME,TRIP_TIME,RISK_MISSED_CONNECTION,NEXT_FLIGHT_TIMELOSS,TOTAL_RISK
0,DL,ATL,"Atlanta, GA",LEX,"Lexington, KY",8/3/2022,5660,910,1018,DL,...,2022-08-03 06:18:00-04:00,2022-08-03 11:30:00-04:00,2022-08-03 12:48:00-04:00,0,0,312.0,458.0,0.706632,866.828876,612.529032
1,DL,ATL,"Atlanta, GA",LEX,"Lexington, KY",8/3/2022,5660,910,1018,DL,...,2022-08-03 06:18:00-04:00,2022-08-03 07:03:00-04:00,2022-08-03 08:25:00-04:00,0,0,45.0,195.0,0.212686,712.051277,151.443233
32,DL,ATL,"Atlanta, GA",DCA,"Washington, DC",8/3/2022,395,730,915,DL,...,2022-08-03 05:15:00-04:00,2022-08-03 08:35:00-04:00,2022-08-03 10:05:00-04:00,0,0,200.0,395.0,0.907326,1197.255655,1086.301447
39,DL,ATL,"Atlanta, GA",EWR,"Newark, NJ",8/3/2022,400,913,1121,DL,...,2022-08-03 07:21:00-04:00,2022-08-03 13:00:00-04:00,2022-08-03 15:01:00-04:00,0,0,339.0,588.0,0.860995,706.521286,608.311068
40,DL,ATL,"Atlanta, GA",EWR,"Newark, NJ",8/3/2022,400,913,1121,DL,...,2022-08-03 07:21:00-04:00,2022-08-03 09:00:00-04:00,2022-08-03 10:49:00-04:00,0,0,99.0,336.0,0.928414,914.954034,849.455816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,DL,ATL,"Atlanta, GA",EWR,"Newark, NJ",8/3/2022,1189,720,927,DL,...,2022-08-03 05:27:00-04:00,2022-08-03 09:00:00-04:00,2022-08-03 10:49:00-04:00,0,0,213.0,449.0,0.049822,961.168467,47.887675
502,DL,ATL,"Atlanta, GA",BHM,"Birmingham, AL",8/3/2022,2377,828,821,DL,...,2022-08-04 03:21:00-05:00,2022-08-04 07:00:00-05:00,2022-08-04 11:00:00-04:00,1,0,219.0,1832.0,0.003771,960.187540,3.620986
503,DL,ATL,"Atlanta, GA",MKE,"Milwaukee, WI",8/3/2022,2446,858,955,DL,...,2022-08-03 04:55:00-05:00,2022-08-03 08:18:00-05:00,2022-08-03 11:26:00-04:00,0,0,203.0,388.0,0.114255,8.243380,0.941847
509,DL,ATL,"Atlanta, GA",BWI,"Baltimore, MD",8/3/2022,2449,715,903,DL,...,2022-08-03 05:03:00-04:00,2022-08-03 08:21:00-04:00,2022-08-03 09:55:00-04:00,0,0,198.0,400.0,0.692489,1239.749169,858.512833
